# Queue historical data calculation via SQS

In [6]:
import boto3
import pandas as pd
import datetime
import pymysql
import os
from urllib.parse import urlparse

In [2]:
# import csv as pandas
df = pd.read_csv('./historical_calculation/cic_start_dates.csv')

In [4]:
# specify last date to calculate
end_date = datetime.datetime.now() - datetime.timedelta(days=1)

# Create SQS client
session = boto3.Session(profile_name='nout_prod')
sqs = session.client('sqs')
queue_url = 'https://sqs.eu-west-1.amazonaws.com/621303523838/sqs_historic_calculation'

for idx, row in df[10:100].iterrows():
    cic_id = row['id']
    first_date = row['first_date']
    date = datetime.datetime.strptime(row['first_date'], '%Y-%m-%d')
    total_days = (end_date - date).days
    i = 1

    while (date < end_date):
        # Send message to SQS queue
        response = sqs.send_message(
            QueueUrl=queue_url,
            DelaySeconds=0,
            MessageAttributes={
                'cic_id': {
                    'DataType': 'String',
                    'StringValue': cic_id},
                'start_date': {
                    'DataType': 'String',
                    'StringValue': date.strftime('%Y-%m-%d')},
                'end_date': {
                    'DataType': 'String',
                    'StringValue': (date+datetime.timedelta(days=1)).strftime('%Y-%m-%d')}
            },
            MessageBody=(
                f'Queuer nr {i} out of total {total_days}.'
            )
        )
        date = date + datetime.timedelta(days=1)
        i += 1
        # print(cic_id, date)

    


In [44]:
df[10:100]

,id,first_date
10,CIC-59e07af0-4dba-565d-86e9-e3d5eede29fa,2022-11-14
11,CIC-56db881a-7d7f-56cf-9077-277aaeb6f949,2022-11-15
12,CIC-7a715ae0-c61f-5a0e-bb76-11944de2e1ef,2022-11-15
13,CIC-a17b541d-ab55-50d9-9352-7412e372165f,2022-11-15
14,CIC-d170ab27-61af-5d53-886b-41082b7376f1,2022-11-15
...,...,...
95,CIC-2d7ede19-2738-5cbc-a718-2be1bfda31f9,2022-12-06
96,CIC-d3ecf2b1-6dbe-5b0e-a10f-22a1a00cc1cb,2022-12-06
97,CIC-d73540e3-b45c-5e01-89ca-296e049be688,2022-12-06
98,CIC-e265a6ef-8365-5bab-a661-c23935c3c6ea,2022-12-06


## manual recalculation of cics with heat production error

In [11]:
mysql_url = os.environ['MYSQLDEV']


parsed_mysql_url = urlparse(mysql_url)
try:
    connection = pymysql.connect(host=parsed_mysql_url.hostname,
                                user=parsed_mysql_url.username,
                                password=parsed_mysql_url.password,
                                database=parsed_mysql_url.path[1:],
                                port=parsed_mysql_url.port,
                                autocommit=True)
    cursor = connection.cursor()
    if cursor.connection:
        try:
            cursor.execute('''SELECT cic_id, date(`time`) as day, quattBuild
                                FROM cic_data
                                WHERE (quattBuild like '0.00.3%' 
                                    or quattBuild like '0.00.1%' 
                                    or quattBuild like '0.00.2%')
                                GROUP BY cic_id, date(`time`)
                                ORDER BY cic_id, date(`time`)''')
            fields = [field_md[0] for field_md in cursor.description]
            result = [dict(zip(fields,row)) for row in cursor.fetchall()]
        except pymysql.Error as err:
            print("Error while executing script {err}", exc_info=True)
except:
    print("Error while connecting to MySQL", exc_info=True)
finally:
    cursor.close()
    connection.close()

In [12]:
df = pd.DataFrame(result)

In [14]:
df

,cic_id,day,quattBuild
0,CIC-049982af-55c4-5af1-bf88-82159e795327,2022-11-17,0.00.23
1,CIC-049982af-55c4-5af1-bf88-82159e795327,2022-11-18,0.00.23
2,CIC-049982af-55c4-5af1-bf88-82159e795327,2023-05-16,0.00.23
3,CIC-05fbece6-7f21-5ab8-bdcd-6670029f1b39,2022-11-14,0.00.10dev
4,CIC-05fbece6-7f21-5ab8-bdcd-6670029f1b39,2022-11-15,0.00.10dev
...,...,...,...
4279,CIC-fedfdcbe-e4a0-5810-9e45-d7377358d3f3,2023-03-25,0.00.26
4280,CIC-fedfdcbe-e4a0-5810-9e45-d7377358d3f3,2023-03-26,0.00.26
4281,CIC-fedfdcbe-e4a0-5810-9e45-d7377358d3f3,2023-03-27,0.00.26
4282,CIC-fedfdcbe-e4a0-5810-9e45-d7377358d3f3,2023-03-28,0.00.26


In [9]:
for idx, row in df.iterrows():
    cic_id = row['id']
    first_date = row['day']
    date = datetime.datetime.strptime(row['first_date'], '%Y-%m-%d')
    i = 1

    # Send message to SQS queue
    response = sqs.send_message(
        QueueUrl=queue_url,
        DelaySeconds=0,
        MessageAttributes={
            'cic_id': {
                'DataType': 'String',
                'StringValue': cic_id},
            'start_date': {
                'DataType': 'String',
                'StringValue': date.strftime('%Y-%m-%d')},
            'end_date': {
                'DataType': 'String',
                'StringValue': (date+datetime.timedelta(days=1)).strftime('%Y-%m-%d')}
        },
        MessageBody=(
            f'[Heat recalculation] Queuer nr {i} out of total {total_days}.'
        )
    )
    i += 1


,cic_id,day,quattBuild
0,CIC-049982af-55c4-5af1-bf88-82159e795327,2022-11-17,0.00.23
1,CIC-049982af-55c4-5af1-bf88-82159e795327,2022-11-18,0.00.23
2,CIC-049982af-55c4-5af1-bf88-82159e795327,2023-05-16,0.00.23
3,CIC-05fbece6-7f21-5ab8-bdcd-6670029f1b39,2022-11-14,0.00.10dev
4,CIC-05fbece6-7f21-5ab8-bdcd-6670029f1b39,2022-11-15,0.00.10dev
...,...,...,...
4279,CIC-fedfdcbe-e4a0-5810-9e45-d7377358d3f3,2023-03-25,0.00.26
4280,CIC-fedfdcbe-e4a0-5810-9e45-d7377358d3f3,2023-03-26,0.00.26
4281,CIC-fedfdcbe-e4a0-5810-9e45-d7377358d3f3,2023-03-27,0.00.26
4282,CIC-fedfdcbe-e4a0-5810-9e45-d7377358d3f3,2023-03-28,0.00.26
